In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly
plotly.tools.set_credentials_file(username='quoniammm', api_key='IF7kV6idFRdoo7LdgGRp')
import plotly.plotly as py
import plotly.graph_objs as go
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

import nltk

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

from keras.preprocessing.sequence import pad_sequences

%matplotlib inline

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')

Using TensorFlow backend.
/home/quoniammm/anaconda3/envs/py3Tfgpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning:

compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6



In [2]:
# bag_words
all_words = set(train['text'].str.split(expand=True).unstack())

def wordandindex(vocab):
    return {word: i + 3 for i, word in enumerate(vocab)}, {i + 3: word for i, word in enumerate(vocab)}

word2index, index2word = wordandindex(all_words)

label_encoder = LabelEncoder()
# 数据集准备
X = np.array(train.text.apply(lambda sen: [word2index[word] for word in sen.split(' ')]))
y = np.array(label_encoder.fit_transform(train.author))
assert len(X) == len(y)
print(len(all_words))
#print(len(train))
print("test length: {}".format(len(test)))

maxlen = 256
X_pad = pad_sequences(sequences=X, maxlen=maxlen)

# 句子填充
# X_pad = np.zeros((19579, 861))

# for i in range(X_pad.shape[0]):
#     for j in range(len(X[i])):
#         X_pad[i, j] = X[i][j]

xtrain, xvalid, ytrain, yvalid = train_test_split(
    X_pad, y, 
    stratify=y, 
    random_state=42, 
    test_size=0.1, 
    shuffle=True
)
print("train length: {}".format(len(xtrain)))
#print(xtrain.type)
print("valid length: {}".format(len(xvalid)))
#print(xvalid.type)

# 最长句子长度设置为 input_size
# max = 0
# for i, x in enumerate(X):
#     # print(len(x))
#     if len(x) > max:
#         max = len(x)
        
# print(max)
# train.iloc[9215].values

epochs = 1
lr = 1e-5

47557
test length: 8392
train length: 17621
valid length: 1958


In [3]:
-

In [4]:
model = textCNN(47560, 256, (3,4,5), 200, 0.6, 3)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.NLLLoss()
model.cuda()

textCNN(
  (embed): Embedding(47560, 256)
  (convs): ModuleList(
    (0): Conv2d (1, 200, kernel_size=(3, 256), stride=(1, 1))
    (1): Conv2d (1, 200, kernel_size=(4, 256), stride=(1, 1))
    (2): Conv2d (1, 200, kernel_size=(5, 256), stride=(1, 1))
  )
  (dropout): Dropout(p=0.6)
  (fc): Linear(in_features=600, out_features=3)
)

In [5]:
-

after 0% training loss=1.5119270086288452, valid loss=1.2965631484985352, at 0 epoch
after 18% training loss=1.1755741834640503, valid loss=0.9429967403411865, at 0 epoch
after 36% training loss=1.1623263359069824, valid loss=1.3072689771652222, at 0 epoch
after 54% training loss=1.234135627746582, valid loss=1.17436945438385, at 0 epoch
after 73% training loss=1.2288755178451538, valid loss=0.8483014702796936, at 0 epoch
after 91% training loss=1.026853084564209, valid loss=1.0943681001663208, at 0 epoch
after 0% training loss=0.8289508819580078, valid loss=1.1814053058624268, at 1 epoch
after 18% training loss=0.9285141229629517, valid loss=0.9456660747528076, at 1 epoch
after 36% training loss=1.1711610555648804, valid loss=1.1111836433410645, at 1 epoch
after 54% training loss=1.3096060752868652, valid loss=1.3559331893920898, at 1 epoch
after 73% training loss=1.1202024221420288, valid loss=1.0608338117599487, at 1 epoch
after 91% training loss=1.0681102275848389, valid loss=1.102

KeyboardInterrupt: 

In [6]:
N = len(train_acc)
print(N)
random_x = np.linspace(0, 1, N)
random_y0 = np.array(train_acc)
random_y1 = np.array(valid_acc)

# Create traces
trace0 = go.Scatter(
    x = random_x,
    y = random_y0,
    mode = 'lines',
    name = 'train_acc'
)
trace1 = go.Scatter(
    x = random_x,
    y = random_y1,
    mode = 'lines',
    name = 'valid_acc'
)

data = [trace0, trace1]

py.iplot(data, filename='line-mode')

240


In [7]:
def test_deal(x):
    sen_vec = []
    for word in x.split(' '):
        if word not in all_words:
            sen_vec.append(2)
        else:
            sen_vec.append(word2index[word])
            
    return sen_vec
            
X_t = np.array(test.text.apply(test_deal))

# X_t_pad = np.zeros((8392, 861))

# for i in range(X_t_pad.shape[0]):
#     for j in range(len(X_t[i])):
#         X_t_pad[i, j] = X_t[i][j]
        
maxlen = 256
X_pad = pad_sequences(sequences=X_t, maxlen=maxlen)
        
tt = Variable(torch.LongTensor(X_t_pad.astype(int)))

In [8]:
model.cpu()
model.eval()

textCNN(
  (embed): Embedding(47560, 128)
  (convs): ModuleList(
    (0): Conv2d (1, 100, kernel_size=(3, 128), stride=(1, 1))
    (1): Conv2d (1, 100, kernel_size=(4, 128), stride=(1, 1))
    (2): Conv2d (1, 100, kernel_size=(5, 128), stride=(1, 1))
  )
  (dropout): Dropout(p=0.6)
  (fc): Linear(in_features=300, out_features=3)
)

In [9]:
for i in range(0, len(tt), 16):
    if i + 16 > len(tt):
        batch = tt[i:]
    else:
        batch = tt[i:i+16]
        
    r, _ = model(batch)
#     print(r)
#     a = np.array(r.data)
#     print(a)
#     b = np.array(r)
#     print(b)
#     c = np.vstack((a, b))
#     print(c)
#     break
    if i == 0:
        out = r.data
        print(out)
    else:
        out = torch.cat((r.data, out), 0)


 0.3850  0.0403  0.5747
 0.5671  0.3251  0.1078
 0.0233  0.9698  0.0070
 0.0492  0.9073  0.0435
 0.2739  0.0224  0.7037
 0.5380  0.0968  0.3652
 0.9907  0.0022  0.0071
 0.0913  0.2509  0.6578
 0.9911  0.0035  0.0054
 0.9953  0.0002  0.0045
 0.0306  0.1016  0.8678
 0.0767  0.8879  0.0354
 0.0089  0.0476  0.9436
 0.0568  0.6581  0.2852
 0.9701  0.0276  0.0022
 0.0127  0.0002  0.9871
[torch.FloatTensor of size 16x3]



In [10]:
len(out)

8392

In [11]:
rr = pd.DataFrame(np.array(out), columns=['EAP', 'HPL', 'MWS'], index=test.id)

In [12]:
rr.to_csv('333.csv')

In [15]:
print("{}".format(round(0.0897*100)/100))

0.09
